# Meta Data Navent Modificado

## ¿Que habia basicamente en los set de datos?
El dataset son propiedades en `venta` en México entre `2012` y `2016`, valuadas en `pesos mexicanos`. El csv de train tiene `240K` filas y `22` columnas. El csv de test tiene `60K` filas (y 21 columnas). La columna a predecir es `precio`.

## Imports 

#### Import common 

In [38]:
import common.common_machine_learning as common

#### Import pandas & numpy

In [39]:
# Importamos librerías de análisis de datos
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

#### Import sklearn 

In [40]:
# Importamos utilidades y modelos de sklearn
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

### Set de datos a usar 

In [41]:
train = common.cargar_set_optimizado('sets_de_datos/train.csv')

## Metrica visualmente conveniente : RMSLE 

## Root Mean Squared Logarithmic Error
$
\huge
\begin{align}
RMSLE = \sqrt{\frac{\sum((log(\text{actual}+1) - log(\text{pred}+1))^2}{n}}
\end{align}
$

In [74]:
from common.metrica import RMSLE

#### Intuición detrás de RMSLE

La razón para usar error logarítmico es que relativiza el error al valor absoluto considerado. Por ejemplo, consideremos un error absoluto de 1000 sobre un valor absoluto de 10 y uno de 100.000:

In [43]:
actual_1 = 10
actual_2 = 1000000
error = 1000

abs_error_1 = np.abs(actual_1 - (actual_1 + error))
abs_error_2 = np.abs(actual_2 - (actual_2 + error))

log_error_1 = np.abs(np.log(actual_1 + 1) - np.log(actual_1 + error + 1))
log_error_2 = np.abs(np.log(actual_2 + 1) - np.log(actual_2 + error + 1))

print(f"Error relativo grande - Abs: {abs_error_1:.4f}, Log:{log_error_1:.4f}")
print(f"Error relativo chico  - Abs: {abs_error_2:.4f}, Log:{log_error_2:.4f}")

Error relativo grande - Abs: 1000.0000, Log:4.5208
Error relativo chico  - Abs: 1000.0000, Log:0.0010


## Métrica Competencia: Mean Absolute Error

In [73]:
from common.metrica import MAE

# Modelos baseline

## Baseline 1: [DummyRegressor](http://scikit-learn.org/stable/modules/model_evaluation.html#dummy-estimators)

Un DummyRegressor(strategy='mean') predice el precio promedio del train set (como constante), no es verdaderamente útil, pero sirve como baseline para darse una idea del bottom-line de valores de predicción

### Preprocesamiento

In [45]:
# Tiramos todas las columnas no numéricas para que sklearn nos acepte el dataframe. 
# Igualmente, el DummyRegressor no va a utilizar ninguna más que el precio 
drop_cols = ['fecha', 'ciudad', 'idzona', 'tipodepropiedad', 'provincia', 'titulo', 'descripcion', 'direccion']
X = train.drop(['precio'] + drop_cols, axis=1)
y = train['precio']
train_X_train, train_X_test, train_y_train, train_y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train de df(train.csv) shapes: X={train_X_train.shape} y={train_y_train.shape}")
print(f"Test de df(train.csv) shapes: X={train_X_test.shape}  y={train_y_test.shape}")

Train de df(train.csv) shapes: X=(180000, 14) y=(180000,)
Test de df(train.csv) shapes: X=(60000, 14)  y=(60000,)


### Modelo

In [46]:
dummy = DummyRegressor(strategy='mean').fit(train_X_train, train_y_train)
pred = dummy.predict(train_X_test)

In [47]:
print(f"Promedio de precios del train de train set: {train_y_train.mean()}")
print(f"Primeras 3 predicciones: {pred[:3]}")

Promedio de precios del train de train set: 2536987.5
Primeras 3 predicciones: [2536913.2 2536913.2 2536913.2]


In [48]:
dummy_rmsle = RMSLE(train_y_test, pred)
dummy_rmsle_train = RMSLE(train_y_train, dummy.predict(train_X_train))
print(f"RMSLE DummyRegressor (train): {dummy_rmsle_train:.5f}")
print('(Error que se comete al predecir la particion de entrenamiento, es decir, datos usados para entrenar el predictor)')
print(f"RMSLE DummyRegressor: {dummy_rmsle:.5f}")
print('(Error que se comete al predecir la particion de test, es decir, nuevos datos)')

RMSLE DummyRegressor (train): 0.90225
(Error que se comete al predecir la particion de entrenamiento, es decir, datos usados para entrenar el predictor)
RMSLE DummyRegressor: 0.90315
(Error que se comete al predecir la particion de test, es decir, nuevos datos)


#### RMSLE no tiene cota superior, asi que 0.903 no es tan malo

In [49]:
dummy_mae = MAE(train_y_test, pred)
dummy_mae_train = MAE(train_y_train, dummy.predict(train_X_train))
print(f"MAE DummyRegressor (train): {dummy_mae_train:.5f}")
print('(Error que se comete al predecir la particion de entrenamiento, es decir, datos usados para entrenar el predictor)')
print(f"MAE DummyRegressor: {dummy_mae:.5f}")
print('(Error que se comete al predecir la particion de test, es decir, nuevos datos)')

MAE DummyRegressor (train): 1612538.75000
(Error que se comete al predecir la particion de entrenamiento, es decir, datos usados para entrenar el predictor)
MAE DummyRegressor: 1602571.25000
(Error que se comete al predecir la particion de test, es decir, nuevos datos)


In [50]:
rmsle_1 = RMSLE(train_y_test, np.array([1 for _ in range(len(train_y_test))]))
rmsle_inf = RMSLE(train_y_test, np.array([9999999999999999999 for _ in range(len(train_y_test))]))
print(f"Si predijesemos todo 1 el RMSLE sería: {rmsle_1:.4f}")
print(f"Si predijesemos un valor altísimo, sería: {rmsle_inf:.4f}")

Si predijesemos todo 1 el RMSLE sería: 13.7364
Si predijesemos un valor altísimo, sería: 29.3569


In [51]:
mae_1 = MAE(train_y_test, np.array([1 for _ in range(len(train_y_test))]))
mae_inf = MAE(train_y_test, np.array([9999999999999999999 for _ in range(len(train_y_test))]))
print(f"Si predijesemos todo 1 el MAE sería: {mae_1:.4f}")
print(f"Si predijesemos un valor altísimo, sería: {mae_inf:.4f}")

Si predijesemos todo 1 el MAE sería: 2512596.5000
Si predijesemos un valor altísimo, sería: 9999999999984371712.0000


## Baseline 2: Regresión lineal sobre metros cubiertos

### Preprocesamiento

In [52]:
X = train[['metroscubiertos']]
y = train['precio']
train_X_train, train_X_test, train_y_train, train_y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train de df(train.csv) shapes: X={train_X_train.shape} y={train_y_train.shape}")
print(f"Test de df(train.csv) shapes: X={train_X_test.shape}  y={train_y_test.shape}")

Train de df(train.csv) shapes: X=(180000, 1) y=(180000,)
Test de df(train.csv) shapes: X=(60000, 1)  y=(60000,)


#### Imputación de nulls
Un [Imputer](http://scikit-learn.org/stable/modules/preprocessing.html#imputation) ("imputador") es una manera de rellenar los nulls  un poco más sofisticada que reemplazarlos por 0 o por -1 (o tirarlos).

En este caso, reemplazamos los nulls por el valor promedio de las muestras existentes, pero podríamos reemplazarlos por otro valor (la moda, la mediana, etc).

Acá imputamos los metros cubiertos.

**OJO!** Al tomar el promedio (`fittear`) hay que hacerlo sobre el train set, sino estaríamos leakeando información!

In [53]:
imp_mean = SimpleImputer()
train_X_train['metroscubiertos'] = imp_mean.fit_transform(train_X_train[['metroscubiertos']])
train_X_test['metroscubiertos'] = imp_mean.transform(train_X_test[['metroscubiertos']])

In [54]:
train_X_test.isnull().sum()

metroscubiertos    0
dtype: int64

### Modelo

In [55]:
linear_model = LinearRegression().fit(train_X_train, train_y_train)
pred = linear_model.predict(train_X_test)

In [56]:
# El modelo de regresion lineal con una variable obtiene RMSLE=0.65
linear_rmsle_train = RMSLE(train_y_train, linear_model.predict(train_X_train))
linear_rmsle = RMSLE(train_y_test, pred)
print(f"RMSLE LinearRegression (train): {linear_rmsle_train:.5f}")
print(f"RMSLE LinearRegression: {linear_rmsle:.5f}")

RMSLE LinearRegression (train): 0.65673
RMSLE LinearRegression: 0.65657


In [57]:
# El modelo de regresion lineal con una variable obtiene RMSLE=0.65
linear_mae_train = MAE(train_y_train, linear_model.predict(train_X_train))
linear_mae = MAE(train_y_test, pred)
print(f"MAE LinearRegression (train): {linear_mae_train:.5f}")
print(f"MAE LinearRegression: {linear_mae:.5f}")

MAE LinearRegression (train): 1202603.98514
MAE LinearRegression: 1191167.10603


## Baseline 3: Árbol de decisión sobre varios features

### Preprocesamiento

In [58]:
# Droppeamos strings y columnas complejas y repetidas
drop_cols = ['titulo', 'descripcion', 'direccion', 'lat', 'lng', 'fecha', 'idzona']
train_simple = train.drop(drop_cols, axis=1).copy()
print(f"Columnas ({len(train_simple.columns)}): {train_simple.columns.tolist()}")
train_simple.head()

Columnas (16): ['id', 'tipodepropiedad', 'ciudad', 'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio']


,id,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,Apartamento,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,False,False,False,False,False,2273000.00
1,53461,Casa en condominio,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,180.00,False,False,False,True,True,3600000.00
2,247984,Casa,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,False,False,False,False,False,1200000.00
3,209067,Casa,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,False,False,False,True,True,650000.00
4,185997,Apartamento,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,False,False,False,False,False,1150000.00


#### Observamos nulls y de features categóricos

In [59]:
display(train_simple.isnull().sum())
numeric_columns_with_nulls = list(set(train_simple.columns[train_simple.isnull().sum() > 0].tolist()) \
                                  - set(['tipodepropiedad', 'ciudad', 'provincia']))
print(numeric_columns_with_nulls)

id                                0
tipodepropiedad                  46
ciudad                          372
provincia                       155
antiguedad                    43555
habitaciones                  22471
garages                       37765
banos                         26221
metroscubiertos               17400
metrostotales                 51467
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
precio                            0
dtype: int64

['antiguedad', 'habitaciones', 'metroscubiertos', 'garages', 'banos', 'metrostotales']


#### One-hot-encodeamos los categóricos [`tipodepropiedad`, `ciudad` y `provincia`] sobre todo el dataset

In [60]:
# Con dummy_na=True, creamos la categoria "Es nulo" como una coordenada más de los one-hot vectors
# Comentar: ¿Hay leaks acá? ¿Sí / No? ¿Por qué?
train_simple = pd.get_dummies(train_simple, dummy_na=True)
print(f"Cantidad de columnas después del one-hot-encoding: {len(train_simple.columns)}")
train_simple.head()

Cantidad de columnas después del one-hot-encoding: 947


,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,tipodepropiedad_Apartamento,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Garage,tipodepropiedad_Hospedaje,tipodepropiedad_Huerta,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Lote,tipodepropiedad_Nave industrial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Terreno industrial,tipodepropiedad_Villa,tipodepropiedad_nan,ciudad_Abalá,ciudad_Abasolo,ciudad_Abejones,ciudad_Acajete,ciudad_Acambay,ciudad_Acaponeta,ciudad_Acapulco de Juárez,ciudad_Acateno,ciudad_Acatic,ciudad_Acatzingo,ciudad_Acayucan,ciudad_Acolman,...,ciudad_Zimapán,ciudad_Zimatlán de Alvarez,ciudad_Zinacantepec,ciudad_Zinapécuaro,ciudad_Zináparo,ciudad_ZirAndaro,ciudad_Zitácuaro,ciudad_Zumpahuacán,ciudad_Zumpango,ciudad_otra,ciudad_ácatlán,ciudad_ácatlán de Juárez,ciudad_ácaxochitlán,ciudad_ácámbaro,ciudad_áutlán de Navarro,ciudad_áyotlán,ciudad_nan,provincia_Aguascalientes,provincia_Baja California Norte,provincia_Baja California Sur,provincia_Campeche,provincia_Chiapas,provincia_Chihuahua,provincia_Coahuila,provincia_Colima,provincia_Distrito Federal,provincia_Durango,provincia_Edo. de México,provincia_Guanajuato,provincia_Guerrero,provincia_Hidalgo,provincia_Jalisco,provincia_Michoacán,provincia_Morelos,provincia_Nayarit,provincia_Nuevo León,provincia_Oaxaca,provincia_Puebla,provincia_Querétaro,provincia_Quintana Roo,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_nan
0,254099,nan,2.00,1.00,2.00,80.00,80.00,False,False,False,False,False,2273000.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,53461,10.00,3.00,2.00,2.00,268.00,180.00,False,False,False,True,True,3600000.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,247984,5.00,3.00,2.00,2.00,144.00,166.00,False,False,False,False,False,1200000.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,209067,1.00,2.00,1.00,1.00,63.00,67.00,False,False,False,True,True,650000.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,185997,10.00,2.00,1.00,1.00,95.00,95.00,False,False,False,False,False,1150000.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### Ejemplo de one-hot-encoding (pasar categóricos a `dummies`)

In [61]:
one_hot_encoding_example_df = pd.DataFrame(['red', 'red', 'green', 'blue', np.nan], columns=['color'])
display(one_hot_encoding_example_df)
display(pd.get_dummies(one_hot_encoding_example_df))
display(pd.get_dummies(one_hot_encoding_example_df, dummy_na=True))

,color
0,red
1,red
2,green
3,blue
4,NaN


,color_blue,color_green,color_red
0,0,0,1
1,0,0,1
2,0,1,0
3,1,0,0
4,0,0,0


,color_blue,color_green,color_red,color_nan
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,1,0,0,0
4,0,0,0,1


Más sobre one-hot-encoding [acá](https://www.kaggle.com/dansbecker/using-categorical-data-with-one-hot-encoding)

#### Imputación de nulls numéricos

In [62]:
# Para los nulls numéricos, usar un Imputer con strategy mean (reemplazamos los NaN por el promedio)
# Para no leakear, spliteamos el dataset antes
X_simple = train_simple.drop("precio", axis=1)
y_simple = train_simple['precio']
train_simple_X_train, train_simple_X_test, train_simple_y_train, train_simple_y_test = train_test_split(
                                                                                            X_simple, 
                                                                                            y_simple, 
                                                                                            test_size=0.25, 
                                                                                            random_state=1)

In [63]:
for c in numeric_columns_with_nulls:
    imp_mean = SimpleImputer()
    train_simple_X_train[c] = imp_mean.fit_transform(train_simple_X_train[[c]])
    train_simple_X_test[c] = imp_mean.transform(train_simple_X_test[[c]])

In [64]:
train_simple_X_train.isnull().sum().sum()

0

In [65]:
train_simple_X_test.isnull().sum().sum()

0

### Modelo

In [66]:
tree = DecisionTreeRegressor().fit(train_simple_X_train, train_simple_y_train)
tree_pred = tree.predict(train_simple_X_test)

In [67]:
tree_rmsle = RMSLE(train_simple_y_test, tree_pred)
tree_rmsle_train = RMSLE(train_simple_y_train, tree.predict(train_simple_X_train))
print(f"RMSLE DecisionTreeRegressor (train): {tree_rmsle_train:.5f}")
print(f"RMSLE DecisionTreeRegressor: {tree_rmsle:.5f}")

RMSLE DecisionTreeRegressor (train): 0.00000
RMSLE DecisionTreeRegressor: 0.47962


In [68]:
tree_mae = MAE(train_simple_y_test, tree_pred)
tree_mae_train = MAE(train_simple_y_train, tree.predict(train_simple_X_train))
print(f"MAE DecisionTreeRegressor (train): {tree_mae_train:.5f}")
print(f"MAE DecisionTreeRegressor: {tree_mae:.5f}")

MAE DecisionTreeRegressor (train): 0.00000
MAE DecisionTreeRegressor: 849212.98907


In [69]:
print(f"RMSLE DummyRegressor       : {dummy_rmsle:.5f}")
print(f"RMSLE LinearRegressor      : {linear_rmsle:.5f}")
print(f"RMSLE DecisionTreeRegressor: {tree_rmsle:.5f}")

RMSLE DummyRegressor       : 0.90315
RMSLE LinearRegressor      : 0.65657
RMSLE DecisionTreeRegressor: 0.47962


In [70]:
print(f"MAE DummyRegressor       : {dummy_mae:.5f}")
print(f"MAE LinearRegressor      : {linear_mae:.5f}")
print(f"MAE DecisionTreeRegressor: {tree_mae:.5f}")

MAE DummyRegressor       : 1602571.25000
MAE LinearRegressor      : 1191167.10603
MAE DecisionTreeRegressor: 849212.98907


# LinearRegressor para submit
El mismo proceso pero con el archivo test.csv en lugar de haciendo train/test split para que tengan el código disponible para generar un submit

In [71]:
train = common.cargar_set_optimizado('sets_de_datos/train.csv', index_col = 0)
test = common.cargar_set_optimizado('sets_de_datos/test.csv', index_col = 0)

In [72]:
# Imputamos los NaNs
imp_mean = SimpleImputer()
train['metroscubiertos'] = imp_mean.fit_transform(train[['metroscubiertos']])
test['metroscubiertos'] = imp_mean.transform(test[['metroscubiertos']])

linear_pred = LinearRegression()\
                    .fit(train[['metroscubiertos']], train['precio'])\
                    .predict(test[['metroscubiertos']])

res = pd.DataFrame(linear_pred, index=test.index, columns=['precio'])
res.columns = ['target']
display(res.head())
res.to_csv("submits/submit_metaDataNavent_mod.csv", header=True) # RMSLE=0.65487

,target
id,
4941,4200756.25
51775,1112322.29
115253,1377423.92
299321,1364168.84
173570,1284638.35
